In [1]:
import networkx as nx
import pandas as pd
import random
import torch
import pickle
from sklearn.model_selection import train_test_split

In [2]:
### import parquet file
unsw_wo_st = pd.read_parquet("C:\\Users\\asus\\Documents\\nids-pcap-dataset\\unsw_parquet_used_dataset\\labeled_wo_start_time.parquet",)

In [7]:
unsw_wo_st = unsw_wo_st[unsw_wo_st['destination_port'] == '80']

In [ ]:
unsw_wo_st['destination_port'] = unsw_wo_st['destination_port'].astype('int32')

In [9]:
unsw_wo_st.is_malware.value_counts()

is_malware
0    109534
1     15811
Name: count, dtype: int64

In [3]:
unsw_wo_st.info(memory_usage='deep')
unsw_wo_st.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 490026 entries, 0 to 490025
Data columns (total 11 columns):
 #   Column            Non-Null Count   Dtype   
---  ------            --------------   -----   
 0   source_ip         490026 non-null  object  
 1   destination_ip    490026 non-null  object  
 2   source_port       490026 non-null  object  
 3   destination_port  490026 non-null  object  
 4   info_message      490026 non-null  object  
 5   attack_category   17093 non-null   category
 6   is_malware        490026 non-null  int32   
 7   source_ip_info    490026 non-null  object  
 8   source_port_info  490026 non-null  object  
 9   dest_ip_info      490026 non-null  object  
 10  dest_port_info    490026 non-null  object  
dtypes: category(1), int32(1), object(9)
memory usage: 350.1 MB


,source_ip,destination_ip,source_port,destination_port,info_message,attack_category,is_malware,source_ip_info,source_port_info,dest_ip_info,dest_port_info
0,149.171.126.18,175.45.176.1,80,4657,HTTP/1.1 200 OK [Malformed Packet],NaN,0,149.171.126.18 HTTP/1.1 200 OK [Malformed Packet],80 HTTP/1.1 200 OK [Malformed Packet],175.45.176.1 HTTP/1.1 200 OK [Malformed Packet],4657 HTTP/1.1 200 OK [Malformed Packet]
1,175.45.176.1,149.171.126.18,4657,80,GET /oKmwKoVbq HTTP/1.1,NaN,0,175.45.176.1 GET /oKmwKoVbq HTTP/1.1,4657 GET /oKmwKoVbq HTTP/1.1,149.171.126.18 GET /oKmwKoVbq HTTP/1.1,80 GET /oKmwKoVbq HTTP/1.1
2,175.45.176.3,149.171.126.18,32473,80,GET /level/15/exec/-/buffers/assigned/dump HTT...,NaN,0,175.45.176.3 GET /level/15/exec/-/buffers/assi...,32473 GET /level/15/exec/-/buffers/assigned/du...,149.171.126.18 GET /level/15/exec/-/buffers/as...,80 GET /level/15/exec/-/buffers/assigned/dump ...
3,175.45.176.2,149.171.126.19,59386,8020,POST /HttpFileUpload/FileUploadController.do H...,NaN,0,175.45.176.2 POST /HttpFileUpload/FileUploadCo...,59386 POST /HttpFileUpload/FileUploadControlle...,149.171.126.19 POST /HttpFileUpload/FileUpload...,8020 POST /HttpFileUpload/FileUploadController...
4,149.171.126.14,175.45.176.0,80,21873,[TCP ACKed unseen segment] HTTP/1.1 200 OK (t...,NaN,0,149.171.126.14 [TCP ACKed unseen segment] HTTP...,80 [TCP ACKed unseen segment] HTTP/1.1 200 OK ...,175.45.176.0 [TCP ACKed unseen segment] HTTP/1...,21873 [TCP ACKed unseen segment] HTTP/1.1 200 ...


In [4]:
unsw_wo_st.is_malware.value_counts()

is_malware
0    472933
1     17093
Name: count, dtype: int64

In [5]:
list_of_words = ["GET", "HTTP"]

In [6]:
info = []
if unsw_wo_st.info_message.str.contains('|'.join(list_of_words)).any():
    info.extend(unsw_wo_st.loc[unsw_wo_st.info_message.str.contains('|'.join(list_of_words)), 'info_message'])

In [7]:
unsw_wo_st = unsw_wo_st[unsw_wo_st.info_message.isin(info)]

In [8]:
unsw_wo_st.is_malware.value_counts()

is_malware
0    471569
1     16889
Name: count, dtype: int64

In [84]:
query_result = unsw_wo_st.query("is_malware == 1").groupby("info_message")['is_malware'].count()

In [85]:
query_result

info_message
GET   / HTTP/1.1                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                6
GET %207U%5d0.dochtml HTTP/1.1                                                                                                                                           

In [86]:
result = unsw_wo_st.groupby("source_ip_info").agg(benign = ('is_malware', lambda x: (x == 0).sum()),
                                                 malware = ('is_malware', lambda x: (x == 1).sum()))

In [87]:
result

,benign,malware
source_ip_info,,
149.171.126.0 HTTP/1.0 200 OK (text/plain),2060,0
149.171.126.0 HTTP/1.1 200 OK (audio/x-ms-wma),173,0
149.171.126.0 HTTP/1.1 200 OK (text/html),15496,0
149.171.126.0 HTTP/1.1 200 OK (video/fli),1850,0
149.171.126.0 HTTP/1.1 404 Not Found,15492,0
...,...,...
59.166.0.9 GET /audio.wma HTTP/1.1,79,0
59.166.0.9 GET /notthere.html HTTP/1.1,4546,0
59.166.0.9 GET /video.fli HTTP/1.1,567,0


In [88]:
result.loc[(result['malware'] == 1) & (result['benign'] == 1)]

,benign,malware
source_ip_info,,
"175.45.176.0 GET / HTTP/1.1 """,1,1
175.45.176.0 GET /%22 HTTP/1.1,1,1
175.45.176.0 GET /%25%25n%25%25n%25%25n%25%25n%25%25n%25%25n%25%25n%25%25n%25%25n%25%25n%25%25n%25%25n%25%25n%25%25n%25%25n%25%25n%25%25n%25%25n%25%25n%25%25n%25%25n%25%25n%25%25n%25%25n%25%25n%25%25n%25%25n%25%25n%25%25n%25%25n%25%25n%25%25n HTTP/1.1,1,1
175.45.176.0 GET /%25%25s%25%25s%25%25s%25%25s%25%25s%25%25s%25%25s%25%25s%25%25s%25%25s%25%25s%25%25s%25%25s%25%25s%25%25s%25%25s%25%25s%25%25s%25%25s%25%25s%25%25s%25%25s%25%25s%25%25s%25%25s%25%25s%25%25s%25%25s%25%25s%25%25s%25%25s%25%25s HTTP/1.1,1,1
175.45.176.0 GET /%2599999999d HTTP/1.1,1,1
...,...,...
175.45.176.3 GET /~ywang/poc.xls HTTP/1.1,1,1
175.45.176.3 POST /OvCgi/jovgraph.exe HTTP/1.1,1,1
175.45.176.3 POST /SiteScope/services/APIBSMIntegrationImpl HTTP/1.1,1,1


#### dump's

In [10]:
trial = unsw_wo_st[['info_message', 'is_malware']].copy()

In [11]:
trial.info()

<class 'pandas.core.frame.DataFrame'>
Index: 125345 entries, 1 to 490022
Data columns (total 2 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   info_message  125345 non-null  object
 1   is_malware    125345 non-null  int32 
dtypes: int32(1), object(1)
memory usage: 2.4+ MB


In [12]:
trial.head()

,info_message,is_malware
1,GET /oKmwKoVbq HTTP/1.1,0
2,GET /level/15/exec/-/buffers/assigned/dump HTT...,0
6,GET eLWfxXSPkc HTTP/1.1,0
9,GET / HTTP/1.1,0
10,GET /scripts/cbag/ag.exe?page=FileDownload&id=...,0


In [13]:
trial['count_benign'] = trial.groupby(['info_message'])['is_malware'].transform(lambda x: (x == 0).sum())
trial['count_malware'] = trial.groupby(['info_message'])['is_malware'].transform(lambda x: (x == 1).sum())

In [14]:
trial['label_fix'] = trial.apply(lambda x: 1 if x['count_malware'] > x['count_benign'] else 0, axis=1)

In [15]:
trial.head()

,info_message,is_malware,count_benign,count_malware,label_fix
1,GET /oKmwKoVbq HTTP/1.1,0,1,0,0
2,GET /level/15/exec/-/buffers/assigned/dump HTT...,0,1,7,1
6,GET eLWfxXSPkc HTTP/1.1,0,1,0,0
9,GET / HTTP/1.1,0,46862,1313,0
10,GET /scripts/cbag/ag.exe?page=FileDownload&id=...,0,1,0,0


In [16]:
import numpy as np

In [17]:
unsw_wo_st['is_malware']  = np.where(unsw_wo_st['info_message'] == trial['info_message'], trial['label_fix'], trial['label_fix'])

In [18]:
unsw_wo_st['count_benign'] = trial['count_benign'].copy()
unsw_wo_st['count_malware'] = trial['count_malware'].copy()

In [ ]:
trial = unsw_wo_st[['info_message', 'is_malware']].copy()
trial['count_benign'] = trial.groupby(['info_message'])['is_malware'].transform(lambda x: (x == 0).sum())
trial['count_malware'] = trial.groupby(['info_message'])['is_malware'].transform(lambda x: (x == 1).sum())
trial['label_fix'] = trial.apply(lambda x: 1 if x['count_malware'] > x['count_benign'] else 0, axis=1)
unsw_wo_st['is_malware']  = np.where(unsw_wo_st['info_message'] == trial['info_message'], trial['label_fix'], trial['label_fix'])
unsw_wo_st['count_benign'] = trial['count_benign'].copy()
unsw_wo_st['count_malware'] = trial['count_malware'].copy()

In [19]:
unsw_wo_st.head()

,source_ip,destination_ip,source_port,destination_port,info_message,attack_category,is_malware,source_ip_info,source_port_info,dest_ip_info,dest_port_info,count_benign,count_malware
1,175.45.176.1,149.171.126.18,4657,80,GET /oKmwKoVbq HTTP/1.1,NaN,0,175.45.176.1 GET /oKmwKoVbq HTTP/1.1,4657 GET /oKmwKoVbq HTTP/1.1,149.171.126.18 GET /oKmwKoVbq HTTP/1.1,80 GET /oKmwKoVbq HTTP/1.1,1,0
2,175.45.176.3,149.171.126.18,32473,80,GET /level/15/exec/-/buffers/assigned/dump HTT...,NaN,1,175.45.176.3 GET /level/15/exec/-/buffers/assi...,32473 GET /level/15/exec/-/buffers/assigned/du...,149.171.126.18 GET /level/15/exec/-/buffers/as...,80 GET /level/15/exec/-/buffers/assigned/dump ...,1,7
6,175.45.176.0,149.171.126.17,49194,80,GET eLWfxXSPkc HTTP/1.1,NaN,0,175.45.176.0 GET eLWfxXSPkc HTTP/1.1,49194 GET eLWfxXSPkc HTTP/1.1,149.171.126.17 GET eLWfxXSPkc HTTP/1.1,80 GET eLWfxXSPkc HTTP/1.1,1,0
9,175.45.176.1,149.171.126.14,51435,80,GET / HTTP/1.1,NaN,0,175.45.176.1 GET / HTTP/1.1,51435 GET / HTTP/1.1,149.171.126.14 GET / HTTP/1.1,80 GET / HTTP/1.1,46862,1313
10,175.45.176.1,149.171.126.19,64694,80,GET /scripts/cbag/ag.exe?page=FileDownload&id=...,NaN,0,175.45.176.1 GET /scripts/cbag/ag.exe?page=Fil...,64694 GET /scripts/cbag/ag.exe?page=FileDownlo...,149.171.126.19 GET /scripts/cbag/ag.exe?page=F...,80 GET /scripts/cbag/ag.exe?page=FileDownload&...,1,0


In [21]:
unsw_wo_st.is_malware.value_counts()

is_malware
0    110294
1     15051
Name: count, dtype: int64

In [22]:
unsw_wo_st = unsw_wo_st.drop(unsw_wo_st[unsw_wo_st.count_benign == unsw_wo_st.count_malware].index)


In [23]:
unsw_wo_st.is_malware.value_counts()

is_malware
0    110274
1     15051
Name: count, dtype: int64

In [ ]:
unsw_wo_st.to_parquet("C:\\Users\\asus\\Documents\\nids-pcap-dataset\\unsw_parquet_used_dataset\\labeled_wo_start_time_w_dup[port].parquet")

In [24]:
nD_unsw_wo_st = unsw_wo_st.drop_duplicates(subset=['dest_port_info'])

In [25]:
nD_unsw_wo_st.is_malware.value_counts()

is_malware
1    8038
0    2252
Name: count, dtype: int64

In [23]:
nD_unsw_wo_st.loc[nD_unsw_wo_st['is_malware'] == 0, 'source_port'].value_counts().sort_values()

source_port
52727         1
51310         1
20209         1
44253         1
44459         1
          ...  
7144         14
8082         14
8028         17
8080         88
80       158304
Name: count, Length: 2138, dtype: int64

In [25]:
nD_unsw_wo_st.loc[(nD_unsw_wo_st['is_malware'] == 1) & (nD_unsw_wo_st['source_port'] == 80), 'source_port'].value_counts().sort_values()

Series([], Name: count, dtype: int64)

In [107]:
nD_unsw_wo_st.reset_index(drop=True, inplace=True)

In [108]:
nD_unsw_wo_st.to_parquet("C:\\Users\\asus\\Documents\\nids-pcap-dataset\\unsw_parquet_used_dataset\\labeled_wo_start_time_no_dup[port].parquet")

In [105]:
value_counts = nD_unsw_wo_st.is_malware.value_counts()
if value_counts[0] > 14926:
    df_to_lower = nD_unsw_wo_st[nD_unsw_wo_st['is_malware'] == 0].sample(n=14926, random_state=13)
    nD_unsw_wo_st = pd.concat([nD_unsw_wo_st[nD_unsw_wo_st['is_malware'] == 1], df_to_lower])

In [74]:
if value_counts[1] > 7463:
    df_to_lower = nD_unsw_wo_st[nD_unsw_wo_st['is_malware'] == 1].sample(n=7463, random_state=13)
    nD_unsw_wo_st = pd.concat([nD_unsw_wo_st[nD_unsw_wo_st['is_malware'] == 0], df_to_lower])

In [75]:
nD_unsw_wo_st.is_malware.value_counts()

is_malware
0    14926
1     7463
Name: count, dtype: int64

#### continue

In [43]:
sum(result['benign']+result['malware'])

488458

In [44]:
train_df, test_df = train_test_split(nD_unsw_wo_st, test_size=0.2, random_state=13)

In [47]:
train_df.is_malware.value_counts()

is_malware
0    11952
1     5959
Name: count, dtype: int64

In [46]:
nD_train_df = train_df.drop_duplicates(subset=['source_ip_info'], keep='first')
nD_train_df.is_malware.value_counts()

is_malware
1    4882
0     969
Name: count, dtype: int64

In [7]:
value_counts = nD_train_df.is_malware.value_counts()
if value_counts[1] > 391:
    df_to_lower = nD_train_df[nD_train_df['is_malware'] == 1].sample(n=391, random_state=72)
    nD_train_df = pd.concat([nD_train_df[nD_train_df['is_malware'] == 0], df_to_lower])

In [8]:
nD_train_df.is_malware.value_counts()

is_malware
0    3916
1     391
Name: count, dtype: int64

In [9]:
test_df.is_malware.value_counts()

is_malware
0    141858
1      5150
Name: count, dtype: int64

In [10]:
nD_test_df = test_df.drop_duplicates(subset=['source_ip_info'])
nD_test_df.is_malware.value_counts()

is_malware
1    3996
0    2227
Name: count, dtype: int64

In [11]:
value_counts = nD_test_df.is_malware.value_counts()
if value_counts[1] > 222:
    df_to_lower = nD_test_df[nD_test_df['is_malware'] == 1].sample(n=222, random_state=72)
    # nD_test_df = pd.concat([nD_test_df, df_to_lower])
    nD_test_df = pd.concat([nD_test_df[nD_test_df['is_malware'] == 0], df_to_lower])

In [12]:
nD_test_df.is_malware.value_counts()

is_malware
0    2227
1     222
Name: count, dtype: int64

In [13]:
nD_test_df.to_parquet('test.parquet')

In [14]:
nD_train_df.to_parquet('train.parquet')

## Building_graph

#### train

In [15]:
nD_graph_train = nx.Graph()
node_features = []
labels = []

for source_ip_info in nD_train_df["source_ip_info"].unique():
    nD_graph_train.add_node(source_ip_info)
    node_features.append([float(len(nD_train_df.info_message))])
    labels.append([float(nD_train_df.is_malware.iloc[0])])
    
for (source_ip), group in nD_train_df.groupby(["source_ip"]):
    for i in range(len(group) - 1):
        from_node = group.iloc[i]["source_ip_info"]
        to_node = group.iloc[i+1]["source_ip_info"]
        if nD_graph_train.has_edge(from_node, to_node):
            nD_graph_train[from_node][to_node]["weight"] += 1
        else:
            nD_graph_train.add_edge(from_node, to_node, weight=1)

In [16]:
node_features_tensor = torch.tensor(node_features)
label_tensor = torch.tensor(labels)
node_features_tensor = node_features_tensor.cuda()
label_tensor = label_tensor.cuda()

In [17]:
torch.save(node_features_tensor, 'graph_modeling/te/node_features_train.pt')
torch.save(label_tensor, 'graph_modeling/te/labels_train.pt')

In [18]:
pickle.dump(nD_graph_train, open('graph_modeling/te/nD_graph_train.pickle', 'wb'))

In [19]:
isolated_nodes = list(nx.isolates(nD_graph_train))

print("Isolated nodes:", len(isolated_nodes), "\n")

Isolated nodes: 0 



#### test

In [20]:
nD_graph_test = nx.Graph()
node_features_test = []
labels_test = []

for source_ip_info in nD_test_df["source_ip_info"].unique():
    nD_graph_test.add_node(source_ip_info)
    node_features_test.append([float(len(nD_test_df.info_message))])
    labels_test.append([float(nD_test_df.is_malware.iloc[0])])

for (source_ip), group in nD_test_df.groupby(["source_ip"]):
    for i in range(len(group) - 1):
        from_node = group.iloc[i]["source_ip_info"]
        to_node = group.iloc[i+1]["source_ip_info"]
        if nD_graph_test.has_edge(from_node, to_node):
            nD_graph_test[from_node][to_node]["weight"] += 1
        else:
            nD_graph_test.add_edge(from_node, to_node, weight=1)

In [21]:
node_features_tensor_test = torch.tensor(node_features_test)
label_tensor_test = torch.tensor(labels_test)
node_features_tensor_test = node_features_tensor_test.cuda()
label_tensor_test = label_tensor_test.cpu()

In [22]:
torch.save(node_features_tensor_test, 'graph_modeling/te/node_features_test.pt')
torch.save(label_tensor_test, 'graph_modeling/te/labels_test.pt')

In [23]:
pickle.dump(nD_graph_test, open('graph_modeling/te/nD_graph_test.pickle', 'wb'))

In [24]:
# Get a list of nodes with no edges
isolated_nodes = list(nx.isolates(nD_graph_test))

# Print the isolated nodes
print("Isolated nodes:", len(isolated_nodes), "\n")

Isolated nodes: 0 



In [25]:
nD_graph_test.number_of_nodes()

2449